# Titanic Space 🛳️🌌🧑‍🚀

In [43]:
import time
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score

In [34]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

### Making preprocessing method

In [35]:
def preprocess_data(df):
    imputer = SimpleImputer(strategy='most_frequent')
    
    for column in ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP']:
        df[column] = imputer.fit_transform(df[[column]]).ravel()
    
    for column in ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']:
        df[column] = imputer.fit_transform(df[[column]])

    df['Deck'] = df['Cabin'].apply(lambda x: x.split('/')[0] if isinstance(x, str) else 'Unknown')
    df['Side'] = df['Cabin'].apply(lambda x: x.split('/')[-1] if isinstance(x, str) else 'Unknown')
    
    df.drop(columns=['Cabin', 'Name'], inplace=True)
    
    le = LabelEncoder()
    for col in ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side']:
        df[col] = le.fit_transform(df[col].astype(str))
    
    return df

### Making model 💪💪💪

In [36]:
train_df = preprocess_data(train_df)
test_df = preprocess_data(test_df)

X = train_df.drop(columns=['PassengerId', 'Transported'])
y = train_df['Transported']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(test_df.drop(columns=['PassengerId']))

model = RandomForestClassifier(random_state=42)

### Supervised learning 📖

In [51]:
X = train_df.drop(columns=['PassengerId', 'Transported'])
y = train_df['Transported']

In [52]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [53]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(test_df.drop(columns=['PassengerId']))

In [54]:
model = RandomForestClassifier(random_state=42)

### Further testing ☢️

In [55]:
start_time = time.time()
model.fit(X_train, y_train)
single_model_time = time.time() - start_time
print(f"Time to train a single model: {single_model_time:.2f} seconds")

Time to train a single model: 1.12 seconds


In [57]:
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

In [47]:
start_time = time.time()
grid_search.fit(X_train, y_train)
grid_search_time = time.time() - start_time
print(f"Total time for GridSearchCV: {grid_search_time / 60:.2f} minutes")

Total time for GridSearchCV: 0.37 minutes


In [48]:
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_val)
print(f'Best Validation Accuracy: {accuracy_score(y_val, y_pred_best)}')

Best Validation Accuracy: 0.7901092581943646


In [49]:
test_predictions = best_model.predict(X_test)

### Creating my submission data file 💫💫💫

In [50]:
submission_df = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Transported': test_predictions
})

submission_df.to_csv('submission.csv', index=False)